In [1]:
# Testing Notebook
# Final code in qkd.py
# Sources: 
# https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/awards/teach_me_qiskit_2018/cryptography/Cryptography.ipynb
# https://qiskit.org/textbook/ch-algorithms/quantum-key-distribution.html

In [2]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute, BasicAer, Aer, transpile, assemble
from qiskit.tools.visualization import plot_histogram
import numpy as np
from numpy.random import randint
import random
import tkinter as tk
import simpy

In [ ]:
# BB84

# Basis-key: {0: Z-Basis (horizontal-vertical), 1: X-Basis (diagonal)}
import wx

app = wx.App()
frame = wx.Frame(parent=None, title="Quantum Key Distribution Simulator")
frame.Show()
app.MainLoop()

In [ ]:
class QKDSIM(wx.Frame):
    
    def __init__(self):
        app = wx.App()   
        super().__init__(parent=None, title="Quantum Key Distribution Simulator")
        panel = wx.Panel(self)        
        my_sizer = wx.BoxSizer(wx.VERTICAL)        
        self.text_ctrl = wx.TextCtrl(panel)
        my_sizer.Add(self.text_ctrl, 0, wx.ALL | wx.EXPAND, 5)        
        my_btn = wx.Button(panel, label='Press Me')
        my_sizer.Add(my_btn, 0, wx.ALL | wx.CENTER, 5)        
        self.Show()
        
app = wx.App()         
frame = QKDSIM()
app.MainLoop()

In [ ]:
# Final code for packaging
im

class QKD:
    
    def __init__(self, seed=None):
        self.seed = seed
            
    def generate_key(n):
        """
        Function for generating a random key and bases of length n.

        :param n: length of bitstring to generate
        """
        if self.seed != None:
            np.random.seed(self.seed)
            
        bit_key = randint(2, size=n)
        bit_basis = randint(2, size=n)

        return bit_key, bit_basis
    
    
    
    

In [ ]:
np.random.seed(seed=42)

In [ ]:
def generate_key(n):
    """
    Function for generating a random key and bases of length n.
    
    :param n: length of bitstring to generate
    """
    bit_key = randint(2, size=n)
    bit_basis = randint(2, size=n)
    
    return bit_key, bit_basis

In [ ]:
n = 100

alice_key, alice_basis = generate_key(n)

print(alice_key)
print(alice_basis)

In [ ]:
def encode(bit_key, bit_bases):
    """
    Function for encoding a message.
    
    :param bit_key: Randomly generated bitstring key
    :param bit_bases: Bases for each bit in bit_key
    """
    output = []
    
    # length of bit_key and bit_bases should be the same
    assert len(bit_key) == len(bit_bases), "Key and bases sequence should be equivalent."
    
    for i in range(len(bit_key)):
        qc = QuantumCircuit(1, 1)
        
        # Encode qubit in Z-basis (horizontal-vertical)
        if bit_bases[i] == 0:
            if bit_key[i] == 0:
                pass
            else:
                qc.x(0)
        
        # Encode qubit in X-basis (diagonal)
        else:
            if bit_key[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
                
        qc.barrier()
        output.append(qc)
        
    return output

In [ ]:
alice_msg = encode(alice_key, alice_basis)

In [ ]:
print("Bit = " + str(alice_key[0]))
print("Basis = " + str(alice_basis[0]))
alice_msg[0].draw()

In [ ]:
def select_basis(msg):
    """
    Function for selecting basis on receiver (Bob) end.
    
    :param msg: Messaged received from sender (Alice)
    """
    return randint(2, size=len(msg))

In [ ]:
# Bob

bob_basis = select_basis(alice_msg)

In [ ]:
def measure(msg, bases, noise=0):
    """
    Function for measure qubits with receiver (Bob) basis.
    
    :param msg: Message received from sender (Alice)
    :param bases: Basis selected by receiver (Bob)
    :param noise: Amount of noise that changes qubit's state
    """
    # Introduce noise 
    if noise > 0:
        
        temp_backend = Aer.get_backend("aer_simulator")
        temp_results = []
        
        noise_idx = []
        for i_n in range(int(len(msg)*noise)):
            noise_idx.append(random.randint(0, len(msg)-1))
            
        for n_idx in noise_idx:
            # Z-basis
            if bases[n_idx] == 0: 
                msg[n_idx].measure(0, 0)

             # X-basis
            if bases[n_idx] == 1:
                msg[n_idx].h(0)
                msg[n_idx].measure(0, 0)
                
            temp_aer_sim = Aer.get_backend("aer_simulator")
            temp_qobj = assemble(msg[n_idx], shots=1, memory=True)
            temp_sim_results = temp_aer_sim.run(temp_qobj).result()
            temp_measured_bit = int(temp_sim_results.get_memory()[0])
            temp_results.append(temp_measured_bit)
         
    backend = Aer.get_backend("aer_simulator")
    results = []
    
    for i in range(len(msg)):
        # Z-basis
        if bases[i] == 0: 
            msg[i].measure(0, 0)
            
         # X-basis
        if bases[i] == 1:
            msg[i].h(0)
            msg[i].measure(0, 0)
            
        aer_sim = Aer.get_backend("aer_simulator")
        qobj = assemble(msg[i], shots=1, memory=True)
        sim_results = aer_sim.run(qobj).result()
        measured_bit = int(sim_results.get_memory()[0])
        results.append(measured_bit)
        
    return results

In [ ]:
bob_key = measure(alice_msg, bob_basis, noise=0.01)
print(bob_key)

In [ ]:
def modify_key(alice_bases, bob_bases, bit_key):
    """
    Function for modifying key by removing bits based on basis matching.
    
    :param alice_bases: Basis that Alice chose
    :param bob_bases: Basis that Bob chose
    :param bit_key: Key to modify
    """
    
    preserved = []
    
    for i in range(len(bit_key)):
        if alice_basis[i] == bob_basis[i]:
            preserved.append(bit_key[i])
        
    return preserved

In [ ]:
alice_mod_key = modify_key(alice_basis, bob_basis, alice_key)
bob_mod_key = modify_key(alice_basis, bob_basis, bob_key)

print("Alice's modified key:\n" + str(alice_mod_key) + "\n")
print("Bob's modified key:\n" + str(bob_mod_key))

In [ ]:
def sifting(alice_mod, bob_mod, sample_size, threshold):
    """
    Function for comparing a random sample of bits in Alice and Bob's keys
    
    :param alice_mod: Alice's current key
    :param bob_mod: Bob's current key
    :param sample_size: Number of bits to compare
    :param threshold: Allowed error proportion
    """
    alice_sample = []
    bob_sample = []
    sample_idx = []
    for i in range(sample_size):
        sample_idx.append(random.randint(0, len(alice_mod)-1))
        
    for idx in sample_idx:
        alice_sample.append(alice_mod[idx])
        bob_sample.append(bob_mod[idx])
    
    error = 1 - (sum(np.array(alice_sample) == np.array(bob_sample)) / 10)
    print("Error rate: " + str(error))
    
    if error <= threshold:
        return alice_mod, bob_mod
    
    else:
        print("Error threshold exceeded. " + str(error*100) + "% discrepancy. \n")
        return None, None

In [ ]:
alice_sifted, bob_sifted = sifting(alice_mod_key, bob_mod_key, sample_size=10, threshold=0)

print("Alice's sifted key:\n" + str(alice_sifted) + "\n")
print("Bob's sifted key:\n" + str(bob_sifted))

# Eavesdropping

In [ ]:
np.random.seed(42)

In [ ]:
n = 100

alice_key, alice_basis = generate_key(n)
alice_msg = encode(alice_key, alice_basis)

In [ ]:
eve_basis = select_basis(alice_msg)

In [ ]:
eve_key = measure(alice_msg, eve_basis) # This function represents eavesdropper interference via measurement

In [ ]:
bob_basis = select_basis(alice_msg)
bob_key = measure(alice_msg, bob_basis)

In [ ]:
alice_mod_key = modify_key(alice_basis, bob_basis, alice_key)
bob_mod_key = modify_key(alice_basis, bob_basis, bob_key)

print("Alice's modified key:\n" + str(alice_mod_key) + "\n")
print("Bob's modified key:\n" + str(bob_mod_key))

In [ ]:
alice_sifted, bob_sifted = sifting(alice_mod_key, bob_mod_key, sample_size=10, threshold=0.03)

print("Alice's sifted key:\n" + str(alice_sifted) + "\n")
print("Bob's sifted key:\n" + str(bob_sifted))

# Risk Analysis

* Chance that Eve guesses the correct basis = 0.5

* Chance that Eve guesses the correct polarization given that she guesses the correct basis = 1.0

* Chance that Eve guesses the wrong basis = 0.5

* Chance that Eve guesses the correct polarization given that she guesses the wrong basis = 0.5

Probability that Eve guesses the correct polarization for one bit = (1.0)(0.5) + (0.5)(0.5) = 0.75

Probability that Eve guesses the correct polarization for all bits = $0.75^{n}$ for n bits

Ex. for 100 bits, P(Eve is discovered) $= 1 - 0.75^{100} = 0.9999999999996793$